## Solving Hangman using probability

### Read file

The file `word_counts.txt` contains the frequency of each 5-letter word from a corpus. The file is formatted as follows: each line contains a word and its frequency, separated by a space. The file is sorted in alphabetical order.

In [1]:
import pandas as pd

# set display precision to 8, default is 6
pd.set_option("display.precision", 8)

word_data = pd.read_csv("word_counts.txt", sep=" ", header=None, names=["w", "count(w)"], dtype={"w":"str", "count(w)":"float64"})
word_data

,w,count(w)
0,AARON,413.0
1,ABABA,199.0
2,ABACK,64.0
3,ABATE,69.0
4,ABBAS,290.0
...,...,...
6530,ZVIAD,30.0
6531,ZWEIG,44.0
6532,ZWICK,34.0
6533,ZYCIE,14.0


### Prior probability

We calculate the prior probabilities of the words by dividing the frequency of each word by the total number of words in the corpus.

In [2]:
word_data["P(w)"] = word_data["count(w)"]/word_data["count(w)"].sum()
word_data

,w,count(w),P(w)
0,AARON,413.0,0.00005388
1,ABABA,199.0,0.00002596
2,ABACK,64.0,0.00000835
3,ABATE,69.0,0.00000900
4,ABBAS,290.0,0.00003784
...,...,...,...
6530,ZVIAD,30.0,0.00000391
6531,ZWEIG,44.0,0.00000574
6532,ZWICK,34.0,0.00000444
6533,ZYCIE,14.0,0.00000183


### Fifteen most frequent 5-letter words

In [3]:
word_data.nlargest(15,"P(w)")

,w,count(w),P(w)
5821,THREE,273077.0,0.03562715
5102,SEVEN,178842.0,0.02333272
1684,EIGHT,165764.0,0.02162650
6403,WOULD,159875.0,0.02085818
18,ABOUT,157448.0,0.02054154
5804,THEIR,145434.0,0.01897413
6320,WHICH,142146.0,0.01854516
73,AFTER,110102.0,0.01436452
1975,FIRST,109957.0,0.01434560
1947,FIFTY,106869.0,0.01394273


### Fourteen least frequent 5-letter words

In [4]:
word_data.nsmallest(14,"P(w)")

,w,count(w),P(w)
712,BOSAK,6.0,0.00000078
895,CAIXA,6.0,0.00000078
3554,MAPCO,6.0,0.00000078
4160,OTTIS,6.0,0.00000078
5985,TROUP,6.0,0.00000078
977,CCAIR,7.0,0.00000091
1107,CLEFT,7.0,0.00000091
1842,FABRI,7.0,0.00000091
2041,FOAMY,7.0,0.00000091
3978,NIAID,7.0,0.00000091


### Best next guess and its probability

In [5]:
def next_guess(word_data:pd.DataFrame, correctly_guessed:str, incorrectly_guessed:list):
    """
    Function that takes the word data (word, count, P(word)), correctly guessed
    and incorrectly guessed as inputs and returns the best next guess and the
    probability of that guess as outputs
    """
    used_letters = set(correctly_guessed.replace("_", "") + "".join(incorrectly_guessed))
    correctly_guessed_letters = set(correctly_guessed.replace("_", ""))
    if len(used_letters):
        probable_words = word_data[word_data["w"].str.match(correctly_guessed.replace("_", "[^" + "".join(used_letters) + "]"))]
    else:
        probable_words = word_data[word_data["w"].str.match(correctly_guessed.replace("_", "."))]
    probable_letters = set("".join(probable_words["w"])) - correctly_guessed_letters

    best_next_guess = ""
    probability_of_next_best_guess = 0
    for l in probable_letters:
        sum_prob = 0
        for w in probable_words["w"]:
            if l in w:
                sum_prob += probable_words.loc[probable_words["w"] == w, "P(w)"].item()/probable_words["P(w)"].sum()
        if sum_prob > probability_of_next_best_guess:
            probability_of_next_best_guess = sum_prob
            best_next_guess = l

    return best_next_guess, probability_of_next_best_guess

### Examples

* `correctly guessed` is the current status of the word, with the letters that have been correctly guessed filled in and the letters that have not been guessed yet replaced with underscores.
* `incorrectly guessed` is the list of letters that have been incorrectly guessed.
* `best next guess` is the best next guess.
* `probability of the guess` is the probability of the best next guess.

In [6]:
inputs = "_____", []
print("correctly guessed: ", inputs[0], ", incorrectly guessed: ", inputs[1])
outputs = next_guess(word_data, *inputs)
print("best next guess:", outputs[0], ", probability of the guess: ", outputs[1])

correctly guessed:  _____ , incorrectly guessed:  []
best next guess: E , probability of the guess:  0.5394172389647987


In [7]:
inputs = "_____", ["E", "A"]
print("correctly guessed: ", inputs[0], ", incorrectly guessed: ", inputs[1])
outputs = next_guess(word_data, *inputs)
print("best next guess:", outputs[0], ", probability of the guess: ", outputs[1])

correctly guessed:  _____ , incorrectly guessed:  ['E', 'A']
best next guess: O , probability of the guess:  0.5340315651557663


In [8]:
inputs = "A___S", []
print("correctly guessed: ", inputs[0], ", incorrectly guessed: ", inputs[1])
outputs = next_guess(word_data, *inputs)
print("best next guess:", outputs[0], ", probability of the guess: ", outputs[1])

correctly guessed:  A___S , incorrectly guessed:  []
best next guess: E , probability of the guess:  0.7715371621621622


In [9]:
inputs = "A___S", ["I"]
print("correctly guessed: ", inputs[0], ", incorrectly guessed: ", inputs[1])
outputs = next_guess(word_data, *inputs)
print("best next guess:", outputs[0], ", probability of the guess: ", outputs[1])

correctly guessed:  A___S , incorrectly guessed:  ['I']
best next guess: E , probability of the guess:  0.7127008416220354


In [10]:
inputs = "__O__", ["A", "E", "M", "N", "T"]
print("correctly guessed: ", inputs[0], ", incorrectly guessed: ", inputs[1])
outputs = next_guess(word_data, *inputs)
print("best next guess:", outputs[0], ", probability of the guess: ", outputs[1])

correctly guessed:  __O__ , incorrectly guessed:  ['A', 'E', 'M', 'N', 'T']
best next guess: R , probability of the guess:  0.7453866259829713


In [11]:
inputs = "_____", ["E", "O"]
print("correctly guessed: ", inputs[0], ", incorrectly guessed: ", inputs[1])
outputs = next_guess(word_data, *inputs)
print("best next guess:", outputs[0], ", probability of the guess: ", outputs[1])

correctly guessed:  _____ , incorrectly guessed:  ['E', 'O']
best next guess: I , probability of the guess:  0.6365554141009603


In [12]:
inputs = "D__I_", []
print("correctly guessed: ", inputs[0], ", incorrectly guessed: ", inputs[1])
outputs = next_guess(word_data, *inputs)
print("best next guess:", outputs[0], ", probability of the guess: ", outputs[1])

correctly guessed:  D__I_ , incorrectly guessed:  []
best next guess: A , probability of the guess:  0.8206845238095237


In [13]:
inputs = "D__I_", ["A"]
print("correctly guessed: ", inputs[0], ", incorrectly guessed: ", inputs[1])
outputs = next_guess(word_data, *inputs)
print("best next guess:", outputs[0], ", probability of the guess: ", outputs[1])

correctly guessed:  D__I_ , incorrectly guessed:  ['A']
best next guess: E , probability of the guess:  0.7520746887966806


In [14]:
inputs = "_U___", ["A", "E", "I", "O", "S"]
print("correctly guessed: ", inputs[0], ", incorrectly guessed: ", inputs[1])
outputs = next_guess(word_data, *inputs)
print("best next guess:", outputs[0], ", probability of the guess: ", outputs[1])

correctly guessed:  _U___ , incorrectly guessed:  ['A', 'E', 'I', 'O', 'S']
best next guess: Y , probability of the guess:  0.6269651101630528
